In [5]:
!pip install qai_hub_models

  Using cached qai_hub_models-0.25.2-py3-none-any.whl.metadata (61 kB)
  Using cached gdown-4.7.1-py3-none-any.whl.metadata (4.4 kB)
  Using cached GitPython-3.1.42-py3-none-any.whl.metadata (12 kB)
  Using cached huggingface_hub-0.23.5-py3-none-any.whl.metadata (12 kB)
  Using cached ipython-8.12.3-py3-none-any.whl.metadata (5.7 kB)
  Using cached opencv_python-4.11.0.86-cp37-abi3-manylinux_2_17_aarch64.manylinux2014_aarch64.whl.metadata (20 kB)
  Using cached prettytable-3.11.0-py3-none-any.whl.metadata (30 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached schema-0.7.5-py2.py3-none-any.whl.metadata (34 kB)
  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
  Using cached qai_hub-0.25.0-py3-none-any.whl.metadata (2.8 kB)
  Using cached datasets-2.14.5-py3-none-any.whl.metadata (19 kB)
  Using cached ruamel.yaml-0.18.10-py3-none-any.whl.metadata (23 kB)
  Using cached dill-0.3.7-py3-none-any.whl.metadata (9.9 kB)
  Using cached

In [4]:
import onnxruntime as ort
import numpy as np
from PIL import Image
import numpy as np


# Step 1: Load the ONNX model
model_path = "/home/gophers/openai_clip-clipimageencoder.onnx"
session = ort.InferenceSession(model_path)


image_path = "/mnt/c/Users/qc_de/Downloads/images/images/669213cb0000000025002493/0.jpg"

def preprocess_image(image_path, target_size=(224, 224), normalize=True):
    # Step 1: Load the image
    img = Image.open(image_path).convert('RGB')  # Convert to RGB if necessary
    
    # Step 2: Resize the image to the target size (height, width)
    img = img.resize(target_size)

    # Step 3: Convert the image to a NumPy array
    img_array = np.array(img)

    # Step 4: Normalize the image (if needed)
    if normalize:
        img_array = img_array.astype(np.float32) / 255.0  # Convert to float32 and scale to [0, 1]

    # Step 5: Change the color format if necessary (HWC -> CHW)
    img_array = np.transpose(img_array, (2, 0, 1))  # (height, width, channels) -> (channels, height, width)
    
    # Step 6: Add a batch dimension (1, channels, height, width)
    img_array = np.expand_dims(img_array, axis=0)  # Shape becomes (1, channels, height, width)
    
    return img_array

# Example usage:
input_data = preprocess_image(image_path)



# Step 2: Prepare input data
# Assuming your model takes an input named 'input' (you can check your model's input name in the ONNX graph)
input_name = session.get_inputs()[0].name


# Example of preparing input data (make sure to match the input shape and type)
# input_data = np.random.random((1, 3, 224, 224)).astype(np.float32)  # Adjust shape and type as needed

# Step 3: Run inference
outputs = session.run(None, {input_name: input_data})

# Step 4: Process the output
# outputs will be a list of results; typically, models have only one output tensor
print("Output:", outputs)

Output: [array([[ 2.69274044e+00, -1.15072279e+01, -4.88524818e+00,
         1.01472979e+01,  1.87115908e+00, -3.70627642e+00,
        -9.24866736e-01, -6.60852337e+00,  2.22608876e+00,
        -2.89952230e+00, -2.31894541e+00, -3.39947701e+00,
        -2.31689334e+00,  7.05896616e-01, -3.62539506e+00,
         6.80639073e-02, -2.51788187e+00, -3.03411722e+00,
         2.89591968e-01,  1.17235076e+00, -2.66956997e+00,
        -4.36455107e+00,  9.33183730e-01,  5.57578135e+00,
         2.56746221e+00,  4.39741135e+00,  5.80170512e-01,
         1.57711029e+00, -3.04702711e+00,  3.56722569e+00,
        -4.25859958e-01,  2.13701820e+00, -1.17988992e+00,
         4.04575443e+00, -5.49516773e+00, -1.20843685e+00,
         1.48400378e+00, -3.86562276e+00,  1.31908188e+01,
         2.99677563e+00,  5.32084799e+00,  1.21513605e+00,
        -1.00234628e+00, -3.35939765e-01, -3.40003061e+00,
         1.57717967e+00,  2.25688720e+00,  8.25984538e-01,
        -1.94588339e+00, -3.12600303e+00,  8.71

In [5]:
loaded_arr = np.load('/home/gophers/array.npy')


In [1]:
import numpy as np
from numpy.linalg import norm

def cosine_similarity(arr1, arr2):
    return np.dot(arr1, arr2) / (norm(arr1) * norm(arr2))

# Example usage:
arr1 = np.array([1, 2, 3])
arr2 = np.array([4, 5, 6])

similarity = cosine_similarity(outputs, loaded_arr)
print(f"Cosine Similarity: {similarity}")


NameError: name 'outputs' is not defined

In [3]:
# ---------------------------------------------------------------------
# Copyright (c) 2024 Qualcomm Innovation Center, Inc. All rights reserved.
# SPDX-License-Identifier: BSD-3-Clause
# ---------------------------------------------------------------------
import argparse
import os

import numpy as np
import torch

from qai_hub_models.models.openai_clip.app import ClipApp
from qai_hub_models.models.openai_clip.model import MODEL_ASSET_VERSION, MODEL_ID, Clip
from qai_hub_models.utils.args import add_output_dir_arg
from qai_hub_models.utils.asset_loaders import CachedWebModelAsset, load_image
from qai_hub_models.utils.display import display_or_save_image


# Run Clip on a directory of images with a query text.
# The demo will display similarity score for each image.
def main(is_test: bool = False):
    # Demo parameters
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--image-dir",
        type=str,
        default=None,
        help="Path to image directory",
    )
    parser.add_argument(
        "--image-names",
        type=str,
        default="image1.jpg,image2.jpg,image3.jpg",
        help="Specify names of the images in the folder.",
    )
    parser.add_argument(
        "--text",
        type=str,
        default="camping under the stars",
        help="Text prompt for image search",
    )
    add_output_dir_arg(parser)
    args = parser.parse_args([] if is_test else None)

    # Load model
    clip_model = Clip.from_pretrained()
    app = ClipApp(clip_model=clip_model)

    # Determine device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Preprocess text
    text = app.process_text(args.text).to(device)

    # Iterate through images and text provided by user
    images = []
    image_names = args.image_names.split(",")
    for i in range(10000):
        for filename in image_names:
            # Make sure the file is an image
            if os.path.splitext(filename)[1].lower() in [".jpg", ".jpeg", ".png"]:
                if not args.image_dir:
                    image = CachedWebModelAsset.from_asset_store(
                        MODEL_ID, MODEL_ASSET_VERSION, filename
                    )
                else:
                    image = os.path.join(args.image_dir, filename)
                # Preprocess image and text pair
                image = app.process_image(load_image(image)).to(device)
                images.append(image)
    
            else:
                print(f"Skipping file {filename}")

    images = torch.stack(images).squeeze(1)

    # Compute similarity
    predictions = app.predict_similarity(images, text).flatten()

    # Display all the images and their score wrt to the text prompt provided.
    print(f"Searching images by prompt: {args.text}")
    for i in range(len(predictions)):
        print(
            f"\t Image with name: {image_names[i]} has a similarity score={predictions[i]}"
        )

    # Show image
    print("Displaying the most relevant image")

    selected_image = image_names[np.argmax(predictions)]
    if not args.image_dir:
        selected_image = CachedWebModelAsset.from_asset_store(
            MODEL_ID, MODEL_ASSET_VERSION, selected_image
        )
    else:
        selected_image = os.path.join(args.image_dir, selected_image)
    most_relevant_image = load_image(selected_image)

    if not is_test:
        display_or_save_image(most_relevant_image, args.output_dir)


if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'qai_hub_models'